# `request`-Modul [requests-Doku](https://requests.readthedocs.io/en/latest/)
* Quickstart unter [Quickstart – resquests-Doku](https://requests.readthedocs.io/en/latest/user/quickstart/)
* Erweiterte Einführung unter [Advanced Usage – requests-Doku](https://requests.readthedocs.io/en/latest/user/advanced/)

In [3]:
import requests

### Diverses
`r = requests.get(...)`

* `r.request.headers`: Header meiner Anfrage


* `r.text`: `HTML`-Code der Seite (?)
* `r.headers`: Header der Antwort
* `r.cookies` oder `r.cookies[<SCHLÜSSEL>]`: Cookies der Antwort/Seite
* `r.raw`: Seiten-Inhalt in Bytes (irgendetwas mit `urllib3`), gibt aber Unterschied zu `r.content`
* `r.content`: Seiten-Inhalt in Bytes, gibt aber Unterschied zu `r.raw`, aber das benötige ich wasl. meistens

## `HTTP`-Requests verschicken
* `r = requests.<REQUEST>(<URL>, ...)`
    `<REQUEST> = (get | post | put | delete | head | options | ... )`
* Rückgabe ist ein ``Response``-Objekt ``r``
* **Immer mit Zeitlimit** verwenden, s. [Timeout/Zeitlimit – requests-Notebook](http://localhost:8888/notebooks/requests_module_Anleitung.ipynb#Timeout/Zeitlimit) (wird hier der Übersicht wegen weggelassen)

In [6]:
r = requests.get('https://api.github.com/events')
r = requests.post('https://httpbin.org/post', data={'key': 'value'})
r = requests.put('https://httpbin.org/put', data={'key': 'value'})
r = requests.delete('https://httpbin.org/delete')
r = requests.head('https://httpbin.org/get')
r = requests.options('https://httpbin.org/get')

### Diverses

* Setzt man `get()`, etc. ab, wird erst ein `Request`-Objekt konstruiert, das zum Server geschickt wird. Nach Erhalt der Antwort wird `Response`-Objekt instanziiert, das das `Request`-Objekt enthält, s. [Prepared Requests – requests-Notebook](http://localhost:8888/notebooks/requests_module_Anleitung.ipynb#Prepared-Requests)
* Body wird direkt heruntergeladen (also, bspw. auch große Dateien). Kann per `stream=True` im Request verhindert werden, sodass dies erst mit Zugriff auf `Response.content` geschieht. Dadurch kann man ggfl. erst entscheiden, ob Body zu groß ist.
    * In diesem Fall können auch `Response.iter_content()` & `Response.iter_lines()` hilfreich sein. Evtl. auch `urllib3.HTTPResponse` bei `Response.raw` (unsicher, was das genau ist, nur der Vollständigkeit wegen aufgeschrieben)
    * Verbindung kann erst geschlossen werden, wenn Daten vollständig konsumiert oder `Response.close()` aufgerufen wurde. Evtl. bietet es sich ein `with`-Konstrukt an (wie bei [`Session` – requests-Notebook](http://localhost:8888/notebooks/requests_module_Anleitung.ipynb#Session-Objekte))
    * [Body Content Workflow – requests-Doku](https://requests.readthedocs.io/en/latest/user/advanced/#body-content-workflow)

In [ ]:
tarball_url = 'https://github.com/psf/requests/tarball/main'
r = requests.get(tarball_url, stream=True)
if int(r.headers['content-length']) < TOO_LONG:
    content = r.content
    ...

### Eigene Header [Custom Headers – requests-Doku](https://requests.readthedocs.io/en/latest/user/quickstart/#custom-headers)

Jede Request-Funktion verfügt über ``headers``-Parameter, der mit einem ``dict``-belegt werden kann

In [6]:
url = 'https://api.github.com/some/endpoint'
headers = {'user-agent': 'my-app/0.0.1'}
r = requests.get(url, headers=headers)  # Schlägt fehl, weil es den "user-agent" bei github nicht gibt

Eigene Header werden in manchen Fällen nachrangig behandelt (s. Quelle)

### Dateien verschicken
Am besten [POST a Multipart-Encoded File – request-Doku](https://requests.readthedocs.io/en/latest/user/quickstart/#post-a-multipart-encoded-file) konsultieren, hier wird der Vollständigkeit halber nur ein Beispiel angeführt.

**Immer** im Binär-Modus `b` öffnen!

In [ ]:
url = 'https://httpbin.org/post'
# **Immer** im Binär-Modus `b` öffnen!
files = {'file': open('report.xls', 'rb')}
r = requests.post(url, files=files)

### URL-Parameter (Pfad- und Queryparameter)
#### Queryparameter
Werden an die URL per KV-Paar ``key=value`` nach einem ``?`` angehängt, bspw. ``httpbin.org/get?key=value``, werden per ``&`` konkateniert, dh ``httpbin.org/get?key=value&key2=value2``.

Per Hand umständlich, deswegen bietet ``requests`` ein ``params``-Argument.

In [7]:
payload = {"key1": "value1", "key2": "value2"}
# Auch mehrere Argumente möglich
# payload = {"key1": "value1", "key2": ["value2", "value3"]}
r = requests.get("https://httpbin.org/get", params=payload)
r.url  # Inspizieren, ob URL richtig zusammengebaut wurde

'https://httpbin.org/get?key1=value1&key2=value2'

#### Pfadparameter
TODO  
...wasl. einfach in die URL schreiben..

### `params`-Attribut & `data`-Attribut
[More complicated POST requests](https://requests.readthedocs.io/en/latest/user/quickstart/#more-complicated-post-requests)

* Möchte `HTML`-Form-Daten übermitteln, muss man das `data`-Attribut verwenden!
    * Mehrere Werte wie oben möglich
* Keine `HTML`-Form-Daten erwünscht: Einfach String verwenden, bspw. `JSON` (s. dazu Link unten) 

Um `HTML`-Form-Daten zu verwenden, muss man wasl. davor auf der Seite, den Namen/die ID des Feldes im Code herausfinden und dann als Schlüssel verwenden.

### Cookies verschicken
Einfach ein `dict` dem `cookies`-Parameter übergeben.

In [10]:
url = 'https://httpbin.org/cookies'
cookies = dict(cookies_are='working')
r = requests.get(url, cookies=cookies)
r.text

'{\n  "cookies": {\n    "cookies_are": "working"\n  }\n}\n'

Cookies werden werden eigentlich per [`RequestCookieJar`](https://requests.readthedocs.io/en/latest/api/#requests.cookies.RequestsCookieJar) übergeben, was eine vollwertigere Schnittstelle für die Behandlung von Cookies darstellt.  
TODO `RequestCookieJar`-Doku durchlesen

In [11]:
jar = requests.cookies.RequestsCookieJar()
jar.set('tasty_cookie', 'yum', domain='httpbin.org', path='/cookies')
jar.set('gross_cookie', 'blech', domain='httpbin.org', path='/elsewhere')
url = 'https://httpbin.org/cookies'
r = requests.get(url, cookies=jar)
r.text

'{\n  "cookies": {\n    "tasty_cookie": "yum"\n  }\n}\n'

### Timeouts/Zeitlimit
* Sollten **immer** verwendet werden, um zu verhindern, dass das Programm unendlich lange wartet!
* `timeout=<ZEIT IN SEK>`, falls es abläuft wird [`Timeout-Exception – requests-Doku`](https://requests.readthedocs.io/en/latest/api/#requests.Timeout) geworfen

In [ ]:
requests.get('https://github.com/', timeout=0.001)

### Prepared Requests [Prepared Requests – requests-Doku](https://requests.readthedocs.io/en/latest/user/advanced/#prepared-requests)
* Möchte man (umfangreichere) Änderungen am Body, Header, etc. vornehmen, bietet sich das `PreparedRequest`-Objekt an (das wird an sich auch in der `Response`-Instanz verwendet)
    * `prepped = req.prepare()`: Einfacher Request
    * `prepped = s.prepare_request(req)`: `Session`-Request, um die Vorteile einer `Session` zu nutzen
* Umgebungsvariablen werden bei dieser Methode ignoriert (bspw. SSL-Fehler, s. Doku-Link oben für mehr Informationen)

In [ ]:
from requests import Request, Session
with Session as s:
    req = Request('POST', url, data=data, headers=headers)
    # Request vorbereiten (evtl. per prepped = s.prepare_request(req))
    prepped = req.prepare()
    # do something with prepped.body
    prepped.body = 'No, I want exactly this as the body.'
    # do something with prepped.headers
    del prepped.headers['Content-Type']
    # Send request
    resp = s.send(
        prepped,
        stream=stream,
        verify=verify,
        proxies=proxies,
        cert=cert,
        timeout=timeout)
    resp.status_code

### Ereignisbedingte Funktionsaufrufe [Event Hooks – requests-Doku](https://requests.readthedocs.io/en/latest/user/advanced/#event-hooks)
* Man kann `hooks` (Funktionen, die bei Erhalt automatisch ausgeführt werden) einer Anfrage/`Session` mitgeben
    * Gegenwärtig sind sie nur bei Erhalt möglich, dh. der einzige Schlüssel ist „`response`“
    * Syntax bei `Session`: `s.hooks["reponse"].append(<FUNKTIONSNAME>)` (bei mehreren einfach Aufruf wiederholen)

In [20]:
def print_url(r, *args, **kwargs):
    print(r.url)

def record_hook(r, *args, **kwargs):
    r.hook_called = True
    return r

# wird Anfrage-URL printen sowie Attribut "hook_called" hinzufügen und auf "True" setzen
r = requests.get('https://httpbin.org/', hooks={'response': [print_url, record_hook]})
f"{r.hook_called = }"

https://httpbin.org/


'r.hook_called = True'

### Eigene Authentifizierung [Custom Authentication – requests-Doku](https://requests.readthedocs.io/en/latest/user/advanced/#custom-authentication)

## ``Response``-Objekt
* ``r.text``: Antwort auf Grundlage von ``r.encoding`` als Text darstellen
    * ``r.encoding`` Gibt Kodierung der Seite an, **kann** falsch sein, deswegen kann man den Wert ändern, sodass ``r.text`` lesbar wird.
    * ``HTTP`` & ``XML`` spezifizieren Encoding im Body, dieser kann per ``r.content`` ausgelesen werden, um dann das Richtige zu setzen.
* ``r.json()`` Antwort als ``JSON``, Mehr unter [JSON REsponse Content – requests-Doku](https://requests.readthedocs.io/en/latest/user/quickstart/#json-response-content)
* ``r.content``: Body der Antwort in Bytes (``b"..."``), bspw können Bilder nicht sinnvoll Text dargestellt werden (mehr unter [Binary Response Content – requests-Doku](https://requests.readthedocs.io/en/latest/user/quickstart/#binary-response-content))
* ``r.raw``: „Raw socket response“ des Servers, dafür ``stream=True`` im initialen Request setzen
    * [Raw Response Content – requests-Doku](https://requests.readthedocs.io/en/latest/user/quickstart/#raw-response-content)
    * ``r.raw.read(<NMB BYTES>)`` um ``NMB BYTES`` auszulesen
    * ...aber lieber ``Response.iter_content()`` nutzen
    * TODO Nicht klar, wofür man das benötigt

### Status-Meldungen/Codes
* `r.status_code` Status-Code der Anwort einholen
    * Es gibt auch „Status-Code-Lookups“, wie `requests.codes.ok` (`== 200`)
* „Fehler“-Status-Codes (`(4|5)\d{2}`) können per `<RESPONSE>.raise_for_status()` geworfen werden.
    * Im Falle eines `r.status_code==200` passiert dann Nichts.

### Antwort-Header [Response Headers – requests-Doku](https://requests.readthedocs.io/en/latest/user/quickstart/#response-headers)
`r.headers` Antwort-Header sind ein `dict` (`HTTP`-Schlüssel sind Kasus-insensitiv)

### Cookies [Cookies – requests-Doku](https://requests.readthedocs.io/en/latest/user/quickstart/#cookies)
`r.cookies[<COOKIE-NAME>]` Zugriff auf Wert des Cookies `<COOKIE-NAME>`

In [ ]:
url = 'http://example.com/some/cookie/setting/url'
r = requests.get(url)
r.cookies['example_cookie_name']

Cookies werden eigentlich als [`RequestsCookieJar – requests-Doku`](https://requests.readthedocs.io/en/latest/api/#requests.cookies.RequestsCookieJar) zurückgegeben und können auch als dieses übergeben werden.  
TODO `RequestCookieJar`-Doku durchlesen

[1]: „.“ in Überschrift, da diese sonst nicht gerendert wird (?)

### Weiterleitung und Historie [Redirection and History – requests-Doku](https://requests.readthedocs.io/en/latest/user/quickstart/#redirection-and-history)
* Weiterleitung aktiviert, außer für `requests.head()`, dann explizit `allow_redirects=True` setzen
* Anzahl der maximalen Weiterleitungen kann angegeben werden
* `r.history: List` Historie der Weiterleitungen inspizieren

In [26]:
# GitHub leitet automatisch von `HTTP` zu `HTTPS` weiter
r = requests.get('http://github.com/')
r.status_code

200

...Aber `r.history` ist nicht leer und zeigt Weiterleitung:

In [27]:
r.history

[<Response [301]>]

* Weiterleitungen können per `allow_redirects=False` unterbunden werden

In [28]:
r = requests.get('http://github.com/', allow_redirects=False)
r.status_code, r.history

(301, [])

## `Session`-Objekte
* Erlaubt bestimmte Parameter über mehrere Requests hinweg zu halten, selbiges gilt für Cookies (nutzt `urllib3`, unterliegende `TCP`-Verbindung wird wiederverwendet)
* Verfügt über alle Methoden eines einfachen Requests (wie `get()`, `post()`, ..., s. [`HTTP`-Request verschicken](http://localhost:8888/notebooks/requests_module_Anleitung.ipynb#HTTP-Requests-verschicken) für mehr)
* `Session`s als Kontext (`with <OBJ> as <NAME>: ...`) verwenden, dann wird Verbindung (auch im Fehlerfall) richtig geschlossen

In [48]:
with requests.Session() as s:
    s.get('https://httpbin.org/cookies/set/sessioncookie/123456789')
    r = s.get('https://httpbin.org/cookies')
    print(r.text.__repr__())

'{\n  "cookies": {\n    "sessioncookie": "123456789"\n  }\n}\n'


* Informationen, die man auf `Session`-Ebene, dh. per `Session`-Objekt `s` setzt bleiben bestehen.  
    * „Informationen“ sind alle möglichen (Wörterbuch-)Daten, die per [`params`](http://localhost:8888/notebooks/requests_module_Anleitung.ipynb#URL-Parameter-(Pfad--und-Queryparameter)), [`data`](http://localhost:8888/notebooks/requests_module_Anleitung.ipynb#params-Attribut-&-data-Attribut), `headers`, `auth`, `cookies` ... übergeben werden
    * `Session`-Wörterbuch-Daten können gelöscht werden, indem der Wert des Schlüssels auf `None` gesetzt wird
* Cookies per [Cookies – requests-Doku](https://requests.readthedocs.io/en/latest/api/#api-cookies) manipulieren/ hinzufügen, etc.

* Informationen auf Methoden-Ebene (hier `headers=...`) werden entweder
    * hinzugefügt oder
    * überschreiben die auf `Session`-Ebene

In [46]:
with requests.Session() as s:
    # Session-Ebene
    s.auth = ('user', 'pass')
    s.headers.update({'x-test': 'true'})
    # Methoden-Ebene; both 'x-test' and 'x-test2' are sent
    r = s.get('https://httpbin.org/headers', headers={'x-test2': 'true'})
    print("x-test" and "x-test2" in r.request.headers)

True


* Methoden-Parameter (hier `cookies=...`) gehen dagegen verloren

In [52]:
with requests.Session() as s:
    r = s.get('https://httpbin.org/cookies', cookies={'from-my': 'browser'})
    print(f"Mit Cookies:\n\t{r.text.__repr__()}")
    r.text
    # '{"cookies": {"from-my": "browser"}}'
    r = s.get('https://httpbin.org/cookies')
    print(f"Ohne Cookies:\n\t{r.text.__repr__()}")
    # '{"cookies": {}}'; Cookie-Informationen der ersten Anfrage nicht dabei

Mit Cookies:
	'{\n  "cookies": {\n    "from-my": "browser"\n  }\n}\n'
Ohne Cookies:
	'{\n  "cookies": {}\n}\n'


## Übersprungen/TODO
* [SSL Cert Verification – requests-Doku](https://requests.readthedocs.io/en/latest/user/advanced/#ssl-cert-verification)
* [Client Side Certificates – requests-Doku](https://requests.readthedocs.io/en/latest/user/advanced/#client-side-certificates)
* [CA Certificates – requests-Doku](https://requests.readthedocs.io/en/latest/user/advanced/#ca-certificates)
* [Streaming Uploads – requests-Doku](https://requests.readthedocs.io/en/latest/user/advanced/#streaming-uploads)
* [Chunk-Encoded Requests – requests-Doku](https://requests.readthedocs.io/en/latest/user/advanced/#chunk-encoded-requests)
* [POST Multiple Multipart-Encoded Files – requests-Doku](https://requests.readthedocs.io/en/latest/user/advanced/#post-multiple-multipart-encoded-files)